In [16]:
#Generating the upload file to MTG tool and Game issue list
import pandas as pd
import os
import numpy as np
import re
        
def upload_file():
    wd = os.getcwd()
    path=os.path.join(wd,'splunk')
    allfile=os.listdir(path)
    files = [os.path.join(path,files) for files in allfile if re.search(r'(\d{2}\.csv$)', files)]

    my_data_0=pd.DataFrame()
    my_data_1=pd.DataFrame()

    for i in files:
        df=pd.read_csv(os.path.join(path,i))
    #df1=df[(df['file'] == 'index.html') | (df['file'] == 'analytics.html')]
        df1=df.loc[df['file'].isin(['index.html','analytics.html'])]
        my_data_0=my_data_0.append(df,sort=False)
        my_data_1=my_data_1.append(df1,sort=False)
        
    file1=os.path.join(path,'Merge.csv')
    file2=os.path.join(path,'MergedFiltered.csv')

    open(file1, 'w').close()
    open(file2, 'w').close()    

    my_data_0.to_csv(os.path.join(path,'Merge.csv'),index=False)
    my_data_1.to_csv(os.path.join(path,'MergedFiltered.csv'),index=False)
    return


def game_sum():
    wd = os.getcwd()
    path=os.path.join(wd,'Initial')
    filelist = os.listdir(path)
    files = [os.path.join(path,files) for files in filelist if files.endswith('.json')]
    mydf=pd.DataFrame()

    for i in files:
    #Read Joson File
        df = pd.read_json(i)
        df=df.loc[:,['id','title', 'issues']].set_index('id')
        df1 = pd.DataFrame([[np.nan] * len(df.columns)], columns=df.columns,index=df.index.unique())

        folder=str(df.index.unique()[0])
        os.mkdir(folder)
    
        mydf=mydf.append(df)
        mydf=mydf.append(df1)
     
    Re = mydf.issues.notna()
    a=mydf['issues'].mask(Re, '"' + mydf['issues'] + '"')
    b = pd.Series('', index=a.index).mask(mydf.issues.shift(-1).notna()& Re, ',')
    mydf['issues_v1'] = (a + b)
    
    new_df=mydf[['title','issues']].dropna().reset_index().set_index(['id','title'])
    counts = new_df.groupby(level=0).cumcount()
    df2=new_df.set_index(counts, append=True).iloc[:,0].unstack()
    
    df2=df2.reset_index()
    df2['id']='MTG_'+df2['id'].astype(str)


    mydf.to_csv('Games For This Week.csv')
    df2.to_csv('Games Issues This Week.csv')
    return
    

upload_file()
#game_sum()

C:\UBS\Dev\Miniconda\lib\site-packages\IPython\core\interactiveshell.py:3191: DtypeWarning: Columns (116,180) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):
C:\UBS\Dev\Miniconda\lib\site-packages\IPython\core\interactiveshell.py:3191: DtypeWarning: Columns (116) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):
C:\UBS\Dev\Miniconda\lib\site-packages\IPython\core\interactiveshell.py:3191: DtypeWarning: Columns (180) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):


In [18]:
#MTG Data Asset Preparation
import pandas as pd
import re
import os
import numpy as np
import shutil
import json

class MTG_Data:
    def __init__(self, path):
         self.path=path
                    

    def csv_create(self):
        files=os.listdir(self.path)
        filename = [filename for filename in files if filename.startswith('Data_GameID')]

        for datafile in filename:
            df=pd.read_csv(datafile)
            name = os.path.splitext(datafile)[0]
            df.to_csv(name+'_Raw'+'.csv',index=False)
            df=df.drop('other_issues',axis=1)
            df.to_csv(datafile,index=False)
        return

    def column_change(self):
        allfiles = os.listdir(self.path)

        game=pd.read_csv('Games For This Week.csv').dropna(axis=0)
        game_list=game['id'].unique().tolist()

        for game_id in game_list:
            filename = [f for f in allfiles if re.search(r'(%s_\d{4}-\d{2}-\d{2}\.csv$)'%(game_id), f)]
            for file in filename:
     
                df=pd.read_csv(file)
                new_name=game.loc[game['id'] == game_id]['issues'].tolist()
                df.rename(columns=dict(zip(df.columns[2:7],new_name)),inplace=True)
                df.to_csv(file,index=False)
        return
    
    def AC_file(self):
        allfiles = os.listdir(self.path)
        
        files = [f for f in allfiles if re.search(r'(-\d{2}\.csv$)', f) and f.startswith('Data_GameID_')]
        open('AC text file.txt', 'w').close()  #Clear the content of existing file

        mydata=pd.DataFrame()
        for filename in files:
            df = pd.read_csv(filename,index_col=0)
            df.rename(columns=dict(zip(df.columns[1:6],['A','B','C','D','E'])),inplace=True)
            mydata=mydata.append(df,sort=False)

        mydata.to_csv(r'AC text file.txt',header=True,sep='\t',mode='a')
        mydata.to_csv('template.csv',index_label='Date_Time')
        return
    
    def replace(self):
        files = os.listdir(self.path)
        filename = [filename for filename in files if filename.startswith('Links_')]
    #os.path.splitext(filename)[0]
        for doc in filename:
            df=pd.read_csv(doc).iloc[0,0]
        
            Game_id=re.search(r'(id=\d{6}&)',df).group()[3:9]
    
            n_filename=re.search(r'(\w{5}_\d{4}-\d{2}-\d{2})',doc).group()\
               +'_'+Game_id+'.csv'

            os.rename(doc,n_filename)
            #print(Game_id)
        return
    
    def add_links(self): 
        files = os.listdir(self.path)
        filename = [filename for filename in files if filename.startswith('Links_')]
        New_value=['Y5135615153446832','X6C79644456466554','X6B7867554F566343','X687749594E315644'\
                   ,'X5747465A46784231','X5646515641435143']
        
        for doc in filename:
            New_list=[]
            df=pd.read_csv(doc).iloc[:,0].tolist()
            Mylink=df[-1] 

            for i in New_value:
                a=Mylink.replace(Mylink[Mylink.index('&ma=')+4:],i)
                New_list.append(a)

            df=pd.DataFrame(df+New_list,columns=['Links']).set_index('Links')

            df.to_csv(doc)
        return
    
    def joson_file(self):
        wd = os.getcwd()
        allfiles = os.listdir(self.path)

        game=pd.read_csv('Games For This Week.csv').dropna(axis=0).set_index('issues')

        results_json = [f for f in allfiles if f.endswith('.json')]

        for i in results_json: 
            with open(i, 'r') as filename:
                json_file=json.load(filename)
                df =pd.DataFrame(json_file['issues'],columns=['issues'])
                data=df.merge(game,on='issues')
                x=data['id'].value_counts().sort_values(ascending=False).index.tolist()[0]
                game_id=str(x)
        #game_id=data.iloc[0,1].astype(str)
       
                try:
                    pos=data.iloc[0,2].index('(')
                except:
                    pos=len(data.iloc[0,2])
            
                new_name=data.iloc[0,2][:pos]

                filename.close()
                os.rename(i,game_id+'.json')
                srcname =os.path.join(wd,game_id+'.json')
                detname=os.path.join(wd,game_id)
                shutil.move(srcname,detname)
         
        
                csv_files = [
                       filename for filename in allfiles if filename.endswith('.csv')
                       and (re.search((r'%s' % game_id), filename))
                ]

                for j in csv_files:
                    srcname_1=os.path.join(wd,j)
                    shutil.move(srcname_1,detname)
         
                shutil.move(game_id,new_name)     
    
    
    
if __name__ == '__main__':  
    mtg=MTG_Data('.')
    mtg.csv_create()
    mtg.column_change()
    mtg.AC_file()
    mtg.replace()
    mtg.add_links()
    mtg.joson_file()    

In [15]:
import pandas as pd
import os
import numpy as np
import re
        
wd = os.getcwd()
path=os.path.join(wd,'splunk')

file1=os.path.join(path,'Merge.csv')
file2=os.path.join(path,'MergedFiltered.csv')


#if os.path.isfile(file1):
open(file1, 'w').close()

#if os.path.isfile(file2):
open(file2, 'w').close()